Vamos a intentar reconstruir el período (`PeriodLS`) de una estrella RRLyrae usando procesos gaussianos.

# Código Auxiliar

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import feets
import feets_patch

from copy import deepcopy

from light_curve.light_curve import LightCurve
from catalog.catalog_loader import CatalogLoader

## Funciones auxiliares

### Graficación

In [2]:
# Auxiliary plotting functions.


# Given a light curve, plots magnitude vs HJD.
def plot_chronological(lc: LightCurve):
    fig, ax = plt.subplots(figsize=(12, 4))

    ax.errorbar(
        lc.time, lc.mag, lc.err, ls="", marker="o", color="tab:blue", ecolor="tab:red"
    )

    ax.set_title(f"Light Curve")
    ax.set_ylabel("Magnitude")
    ax.set_xlabel("HJD")

    ax.invert_yaxis()
    fig.tight_layout()


# Given a periodic light curve, plots magnitude vs period in two phases.
def plot_periodic(lc: LightCurve):
    # duplicate the values in two phases
    phase = np.hstack((lc.phase, lc.phase + 1))
    pmag = np.hstack((lc.pmag, lc.pmag))
    perr = np.hstack((lc.perr, lc.perr))

    # plot the folded light curve
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.errorbar(
        phase, pmag, perr, ls="", marker="o", ecolor="tab:blue", color="tab:red"
    )
    ax.set_title(f"Folded Light Curve")
    ax.set_ylabel("Magnitude")
    ax.set_xlabel("Phase")
    ax.invert_yaxis()

    fig.tight_layout()

    return ax

# Obtención de los datos

In [3]:
# Get the catalogs.
loader = CatalogLoader("../catalog")
b278_lc = loader.get_lc("b278")
b278_features = loader.get_features("b278")

# Filter RRLyraes from the catalogs.
periodic_star_types = ["RRLyr-RRab", "RRLyr-RRc", "RRLyr-RRd"]
b278_features = b278_features[b278_features["vs_type"].isin(periodic_star_types)]
b278_lc = b278_lc[b278_lc.bm_src_id.isin(b278_features.id.to_numpy())]

# Select a star.
star = b278_features.iloc[8]

# Get the observations for the star.
light_curve = b278_lc[b278_lc.bm_src_id == star.id]

lc = LightCurve(light_curve, star.PeriodLS)

# Reconstrucción de Período

Ahora el experimento. Vamos a elegir `n_sample` puntos de la curva de luz al azar y vamos a poner esos puntos en fase usando Lomb-Scargle. Luego vamos a hacer `n_iter` veces lo siguiente:
* Entrenar un proceso gaussiano con los puntos.
* Utilizarlo para agregar `n_synthetic` puntos que se encuentran en un valor al azar (uniformemente distribuido) de la fase.
* Utilizar todos los puntos para recalcular el período usando Lomb-Scargle.
  
Queremos ver si logramos acercarnos al período del catálogo.

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
# Parameters.
n_sample = 40
n_iter = 20
n_synthetic = 1

In [6]:
# Select a star.
star = b278_features.iloc[9]

# Get the observations for the star.
light_curve = b278_lc[b278_lc.bm_src_id == star.id]
synthetic_lc = LightCurve(light_curve, star.PeriodLS, star.id)
synthetic_lc.filter_snr(20)

In [7]:
period_list = []
period_fit_list = []

# synthetic_lc.subsample(n_sample)

for _ in range(n_iter):
    synthetic_lc.add_synthetic(n_synthetic)
    period, period_fit = synthetic_lc.make_periodic()
    period_list.append(period)
    period_fit_list.append(period_fit)

## Resultados

In [8]:
period_list

[0.8860660646342517,
 0.319340839325383,
 0.319340839325383,
 0.886225930489819,
 0.2612666999803157,
 0.2612666999803157,
 0.2612666999803157,
 0.17339863551621293,
 0.2612666999803157,
 0.2612666999803157,
 0.2612666999803157,
 0.2612666999803157,
 0.2612666999803157,
 0.14778119560551853,
 0.14778119560551853,
 0.10000619418185638,
 0.10000517614255815,
 0.10000517614255815,
 0.10000517614255815,
 0.10000517614255815]

In [9]:
period_fit_list

[0.5396094695933689,
 0.9886995324174719,
 0.9601198524574246,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]